# Practice 1: Data Science Salaries
In this practice assignment I looked at the Diabetes prediction dataset provided by Mohammed Mustafa on [kaggle](https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset) and predicted the diabetes. 

The following questions will be anwsered in this document:
* What is unbalanced data?
* What is a ROC curve?
* WHat is the impact of unbalanced data on the results and how to mitigate it?
* What is the implication of using the random sampler, should it be sampled before or after dividing the data into a train and test set?
* What happens if the model trained on the sampled data is applied on the entire dataset?

## Made By Jarno Weemen

In [2]:
# Neccessary
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df_diabetes = pd.read_csv("Data/diabetes_prediction_dataset.csv")
df_diabetes.head(10)

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
5,Female,20.0,0,0,never,27.32,6.6,85,0
6,Female,44.0,0,0,never,19.31,6.5,200,1
7,Female,79.0,0,0,No Info,23.86,5.7,85,0
8,Male,42.0,0,0,never,33.64,4.8,145,0
9,Female,32.0,0,0,never,27.32,5.0,100,0


# Looking at data
First of all lets take a quick look at the data to see if it needs cleaning.

In [6]:
df_diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


There doesn't seem to be any values that are null or have a weird data type.

In [7]:
for col in df_diabetes.columns:
    print("Col: {}, Unique values: {}".format(col, df_diabetes[col].unique()))

Col: gender, Unique values: ['Female' 'Male' 'Other']
Col: age, Unique values: [80.   54.   28.   36.   76.   20.   44.   79.   42.   32.   53.   78.
 67.   15.   37.   40.    5.   69.   72.    4.   30.   45.   43.   50.
 41.   26.   34.   73.   77.   66.   29.   60.   38.    3.   57.   74.
 19.   46.   21.   59.   27.   13.   56.    2.    7.   11.    6.   55.
  9.   62.   47.   12.   68.   75.   22.   58.   18.   24.   17.   25.
  0.08 33.   16.   61.   31.    8.   49.   39.   65.   14.   70.    0.56
 48.   51.   71.    0.88 64.   63.   52.    0.16 10.   35.   23.    0.64
  1.16  1.64  0.72  1.88  1.32  0.8   1.24  1.    1.8   0.48  1.56  1.08
  0.24  1.4   0.4   0.32  1.72  1.48]
Col: hypertension, Unique values: [0 1]
Col: heart_disease, Unique values: [1 0]
Col: smoking_history, Unique values: ['never' 'No Info' 'current' 'former' 'ever' 'not current']
Col: bmi, Unique values: [25.19 27.32 23.45 ... 59.42 44.39 60.52]
Col: HbA1c_level, Unique values: [6.6 5.7 5.  4.8 6.5 6.1 6.  5.

All of the unique values also look good, even the age where you have . values, because a baby of 1 week and 50 weeks is already quite the difference.

# Analysing the data
Now that we know the data is ready for use, we can look at a couple of things:
* Features of people with and without diabetes.
* Potential outliers.
* ROC curve